# Part 6:  Codes secrets et échange de clés en Python 3.x

Dans ce notebook, nous introduisons la cryptographie -- comment communiquer en toute sécurité sur des canaux non sécurisés. Nous commençons par l’étude de deux méthodes simples de code secret (appelé aussi chiffrement), le code César et sa variante la plus sophistiquée, le code Vigenère. Le code Vigenère utilise une **clé** pour transformer le **texte en clair** (c’est-à-dire le message) en **texte chiffré** (le message codé) et utilise la même clé pour transformer le texte chiffré en texte en clair. Par conséquent, deux parties peuvent communiquer en toute sécurité si elles - et elles seules - possèdent la clé.

Si la sécurité de la communication repose sur la possession d'une clé commune, il nous reste un nouveau problème: comment les deux parties s'accordent-elles sur une clé commune, surtout si elles sont éloignées l'une de l'autre et communiquent par un canal peu sécurisé?

Une solution intelligente à ce problème a été publiée en 1976 par Whitfield Diffie et Martin Hellman, et s'appelle donc échange de clés Diffie-Hellman. Elle tire parti de l'arithmétique modulaire et repose sur l'existence d'une racine primitive (modulo un nombre premier) et sur la difficulté à résoudre le **problème du logarithme discret**.

## Table des matières

- [Code secret](#cipher)
- [Echange de clés](#keyexchange)


<a id='cipher'></a>

## Codes

Un chiffrement est un moyen de transformer un message, appelé **texte en clair**, en une forme différente, le **texte chiffré**, qui masque le sens à tout le monde, sauf au destinataire. Par exemple, dans un **code par substitution** on change chaque lettre de l'alphabet en une autre lettre. Ce n'est pas trop difficile à résoudre à la main et c'est facile à résoudre pour les ordinateurs, grâce à l’analyse de fréquence (compréhension de la fréquence à laquelle différentes lettres ou combinaisons de lettres se produisent).


### Caractères ASCII et le code César

Même si les codes par substitution sont faciles à briser, ils constituent un bon point de départ. Pour implémenter des chiffrements par substitution en Python, nous devons étudier le type *string* (chaine de caractères) plus en détail. Pour déclarer une chaîne de caractères, il suffit de mettre la chaîne entre guillemets. Vous pouvez utiliser des lettres, des chiffres, des espaces et de nombreux symboles dans une chaîne. Vous pouvez entourer votre chaîne de caractères par des guillemets simples, comme `'Hello'` ou des guillemets doubles, comme `"Bonjour"`. Cette flexibilité est pratique si vous souhaitez utiliser des guillemets dans la chaîne. Par exemple, la chaîne *C'est un nombre premier* devrait être décrite en Python avec des guillemets `"C'est un nombre premier"` afin que l'apostrophe ne se confonde pas avec les guillemets.

Les chaînes sont indexées comme des listes de caractères.

In [ ]:
W = "Hello"
print(W)
for j in range(len(W)):  # len(W) is the length of the string W.
    print(W[j])  # Access the jth character of the string.

 Un caractère est encore de type string ; c'est une chaîne de longueur 1.

In [ ]:
print(type(W))
print(type(W[0]))  # W[0] is a character.

Comme les ordinateurs stockent les données en binaire, les concepteurs des premiers ordinateurs (dans les années 1960) ont créé un code appelé [**ASCII**](https://en.wikipedia.org/wiki/ASCII) (American Standard Code for Information Interchange) qui associe à chaque caractère un nombre compris entre 0 et 127. Chaque nombre compris entre 0 et 127 est représenté en binaire par 7 bits (entre 0000000 et 1111111), ainsi chaque caractère est stocké avec 7 bits de mémoire. Plus tard, l'ASCII a été étendu avec 128 caractères supplémentaires, de sorte que les codes compris entre 0 et 255 ont été utilisés, nécessitant 8 bits. Une suite de 8 bits de mémoire est appelée **octet**. Un octet de mémoire suffit pour stocker un caractère ASCII étendu.

Vous remarquerez peut-être que 256 caractères ASCII sont disponibles, mais votre clavier compte moins de 256 caractères, même une fois que vous avez inclus des symboles tels que `#` et `;`. Certains de ces caractères supplémentaires sont destinés aux lettres accentuées, tandis que d'autres sont des reliques d'anciens ordinateurs. Si vous êtes curieux, vous pouvez rechercher un [tableau ASCII complet](http://www.asciitable.com/).

De nos jours, la communauté mondiale des utilisateurs nécessite bien plus que 256 caractères - il existe de nombreux alphabets dans le monde! Ainsi, au lieu d’ASCII, nous pouvons accéder à plus de 100 000 **caractères Unicode**. Faites défiler une [table unicode](https://unicode-table.com/en/) pour voir ce qui est possible. Avec emoji, les tables unicode sont entrées dans un [territoire inexploré](https://xkcd.com/1813/). Python version 3.x prend totalement en charge le format Unicode.

Mais ici, nous restons dans le code ASCII à l’ancienne, car il suffit pour les messages de base en anglais. Python a les commandes intégrées `chr` et` ord` pour la conversion numéro (0--255) en caractère et inversement.

In [ ]:
chr(65)

In [ ]:
ord('A')

Le code suivant produira une table des caractères ASCII avec des numéros compris entre 32 et 126. Il s’agit d’une plage qui inclut tous les caractères et symboles anglais les plus courants sur un clavier US. Notez que le numéro ASCII 32 correspond à un espace vide (un caractère important pour les longs messages!)

In [ ]:
for a in range(32,127):
    c = chr(a)
    print("ASCII {} is {}".format(a, c))

Puisque nous travaillerons uniquement avec la plage ASCII comprise entre 32 et 126, il sera utile de "passer" d'autres nombres dans cette plage. Par exemple, nous interpréterons 127 comme 32, 128 comme 33, etc., lorsque nous convertirons des nombres en dehors de la plage considérée.

La fonction suivante force un nombre dans une plage donnée, en utilisant l'opérateur mod. C'est une astuce courante: faire en sorte que les listes tournent cycliquement.

In [ ]:
def inrange(n,range_min, range_max):
    '''
    The input number n can be any integer.
    The output number will be between range_min and range_max (inclusive)
    If the input number is already within range, it will not change.
    '''
    range_len = range_max - range_min + 1
    a = n % range_len
    if a < range_min:
        a = a + range_len
    return a

In [ ]:
inrange(13, 1, 10)

In [ ]:
inrange(17, 5, 50)

Nous pouvons maintenant mettre en place un **chiffrement par substitution** en convertissant les caractères en leurs numéros ASCII, en mélangeant les numéros et en reconvertissant. L'un des plus simples algorithmes de substitution s'appelle **code César**, dans lequel chaque caractère est décalé d'un montant fixe. Par exemple, un code César avec décalage 3 enverrait «A» à «D» et «B» à «E», etc. À la fin de la liste les caractères sont décalés cycliquement, en utilisant la fonction `inrange`.

#### Exercice
Ecrire une fonction `Caesar_shift`, qui prend un caractère ASCII `c` (dont le numéro est compris entre 32 et 126) et qui renvoie le caractère décalé, au sens du code César, du montant `shift`.

In [ ]:
def Caesar_shift(c, shift):
    '''
    Shifts the character c by shift units
    within the ASCII table between 32 and 126. 
    The shift parameter can be any integer!
    '''
    # code de la fonction Caesar_shift ici

Voyons l'effet du code de César sur notre table ASCII.

In [ ]:
for a in range(32, 127):
    c = chr(a)
    print("ASCII {} is {}, which shifts to {}".format(a, c, Caesar_shift(c, 5))) # Shift by 5.

#### Exercice
Ecrire une fonction `Caesar_cipher` qui prend en arguments une chaine de caractères `plaintext`, le message en clair, ainsi qu'un entier `shift`, et qui renvoie le message chiffré par un code César de décalage `shift`.

In [ ]:
def Caesar_cipher(plaintext, shift):
    # code de la fonction Caesar_cipher ici

In [ ]:
print(Caesar_cipher('Hello! Can you read this?', 5))  # Shift forward 5 units in ASCII.

Comme prévu, le chiffrement Caesar transforme le texte en clair en texte chiffré, en utilisant un décalage de la table ASCII.
Pour déchiffrer le texte chiffré, on utilise à nouveau le code César, avec le décalage négatif.

In [ ]:
print(Caesar_cipher('Mjqqt&%%Hfs%~tz%wjfi%ymnxD', -5))  # Shift back 5 units in ASCII.

### Le code Vigenère

Le chiffrement de César est assez facile à casser, par une attaque en force brute (décalage de toutes les valeurs possibles) ou une analyse de fréquence (comparez la fréquence des caractères dans un message à la fréquence des caractères dans les messages anglais typiques).
Le chiffrement Vigenère est une variante du chiffrement Caesar qui utilise une **clé de chiffrement** pour faire varier le paramètre de décalage tout au long du processus de chiffrement. Par exemple, pour chiffrer le message "This is very secret" à l'aide de la clé "Key", vous devez aligner les caractères du message au-dessus des copies répétées de la clé.

T | h | i | s |   | i | s |   | v | e | r | y |   | s | e | c | r | e | t
--|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|--
K | e | y | K | e | y | K | e | y | K | e | y | K | e | y | K | e | y | K

Ensuite, vous transformez tout en ASCII et utilisez la rangée inférieure pour décaler la rangée supérieure.

ASCII message | 84 | 104 | 105 | 115 | 32 | 105 | 115 | 32 | 118 | 101 | 114 | 121 | 32 | 115 | 101 | 99 | 114 | 101 | 116 
---|-----|-----
Shift | 75 | 101 | 121 | 75 | 101 | 121 | 75 | 101 | 121 | 75 | 101 | 121 | 75 | 101 | 121 | 75 | 101 | 121 | 75 
ASCII shifted | 159 | 205 | 226 | 190 | 133 | 226 | 190 | 133 | 239 | 176 | 215 | 242 | 107 | 216 | 222 | 174 | 215 | 222 | 191 
ASCII shifted in range |   64 | 110 | 36 | 95 | 38 | 36 | 95 | 38 | 49 | 81 | 120 | 52 | 107 | 121 | 32 | 79 | 120 | 32 | 96 

Enfin, les numéros ASCII décalés sont reconvertis en caractères pour la transmission. Dans ce cas, les numéros 64, 110, 36, 95, etc., sont convertis en texte chiffré ``"@n$_&$_&1Qx4ky Ox \`"``

Le chiffre de Vigenère est beaucoup plus difficile à déchiffrer que le chiffre de César, si vous n'avez pas la clé. En effet, les variations de décalage rendent l'analyse de fréquence plus difficile. 

Le chiffrement de Vigenère est cependant considéré comme faible avec les normes actuelles 
(voir [Wikipedia](https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher#Cryptanalysis) pour une description des attaques au 19ème siècle), mais il illustre les ingrédients de base d'un cryptosystème avec **clé symétrique** : le texte en clair, le texte chiffré et une clé unique. Aujourd'hui, les cryptosystèmes à clé symétrique tels que 
[AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) et 
[3DES](https://en.wikipedia.org/wiki/Triple_DES) sont utilisés en permanence pour la communication sécuririsée.

#### Exercice
Ecrire une fonction `Vigenere_cipher` qui prend en arguments un message, en clair, et une clé, et qui renvoie le message codé par Vigenère.

In [ ]:
def Vigenere_cipher(plaintext, key):
    # code fonction Vigenere_cipher ici

In [ ]:
print(Vigenere_cipher('This is very secret', 'Key')) # 'Key' is probably a bad key!!

Le chiffrement de Vigenère est appelé cryptosystème **symétrique**, car la même clé que celle utilisée pour chiffrer le texte en clair peut être utilisée pour déchiffrer le texte chiffré.

#### Exercice
Ecrire une fonction `Vigenere_decipher` qui prend en arguments un message codé par Vigenère, ainsi que la clé, et qui renvoie le message décodé.

In [ ]:
def Vigenere_decipher(ciphertext, key):
    # code fonction Vigenere_decipher ici

In [ ]:
print(Vigenere_decipher('@n$_&$_&1Qx4ky Ox `', 'Key'))

In [ ]:
# Try a few cipher/deciphers yourself to get used to the Vigenere system.
# Essayez quelques cipher/deciphers par vous-même pour vous habituer au cryptosystème Vigenere.

Le chiffrement Vigenère devient un moyen efficace pour une communication sécurisée entre deux parties, à condition de partager une clé secrète. Au 19ème siècle, cela signifiait souvent que les parties nécessitaient une première réunion en personne pour se mettre d'accord sur une clé, ou qu'un messager bien gardé la porterait d'une partie à l'autre.

Aujourd'hui, comme nous souhaitons communiquer régulièrement et en toute sécurité sur de longues distances, il est plus difficile de s'entendre sur une clé. Cela ressemble au problème de la poule et de l'œuf, où nous avons besoin d'un secret partagé pour communiquer de manière sécurisée, mais nous ne pouvons pas partager un secret sans communiquer de manière sécurisée en premier lieu!

Il est remarquable que ce problème de partage de secrets peut être résolu avec quelques astuces d'arithmétique modulaire. C'est le sujet de la section suivante.

### Exercises

1. Un chiffrement César a été utilisé pour coder un message, avec le texte chiffré suivant: `'j!\'1r$v1"$v&&+1t}v(v$2'`. 
Utilisez une boucle (attaque par force brute) pour déterminer le message d'origine.

2. Imaginez que vous chiffriez un long message (par exemple 1000 mots d’anglais standard) avec un chiffrement Vigenère. Comment pourriez-vous détecter la *longueur* de la clé, si elle est courte (par exemple 3 ou 4 caractères)?

3. Envisagez de coder un message en texte clair par Vigenère avec une clé à 3 caractères, puis recoder le texte chiffré avec une clé à 4 caractères. Expliquez en quoi cela équivaut à coder le message d'origine via un seul chiffrement avec une clé de 12 caractères.

<a id='keyexchange'></a>

## Echange de clés

Nous étudions maintenant l’échange de clés **Diffie-Hellman**, un moyen remarquable pour deux parties de partager un secret sans jamais avoir besoin de le communiquer directement. Leur méthode est basée sur les propriétés d’exponentiation modulaire et l’existence d’une **racine primitive** modulo un nombre premier.

### Racines primitives et nombres premiers de Sophie Germain

Si $p$ est un nombre premier et $GCD(a, p) = 1$, le petit théorème de Fermat dit que 
$$a^{p-1} \equiv 1 \text{ mod } p$$
Il peut cependant arriver que  $a^\ell \equiv 1$ mod $p$ pour une plus petite valeur (positive) de $\ell$. La plus petite valeur positive de $\ell$ est appelée l'**ordre** (ordre multiplicatif, pour être précis) de $a$ modulo $p$, et il s'agit toujours d'un diviseur de $p-1$.

Le code suivant détermine l'ordre d'un nombre, mod $p$, avec une approche par force brute.

In [1]:
def mult_order(a, p):
    '''
    Determines the (multiplicative) order of an integer
    a, modulo p.  Here p is prime, and GCD(a,p) = 1.
    If bad inputs are used, this might lead to a 
    never-ending loop!
    '''
    current_number = a % p
    current_exponent = 1
    while current_number != 1:
        current_number = (current_number * a)%p
        current_exponent = current_exponent + 1
    return current_exponent
    

In [11]:
p = 11
for j in range(1, p):
    print("The multiplicative order of {} modulo {} is {}".format(j, p, mult_order(j, p)))
    # These orders should all be divisors of p-1.

The multiplicative order of 1 modulo 11 is 1
The multiplicative order of 2 modulo 11 is 10
The multiplicative order of 3 modulo 11 is 5
The multiplicative order of 4 modulo 11 is 5
The multiplicative order of 5 modulo 11 is 5
The multiplicative order of 6 modulo 11 is 10
The multiplicative order of 7 modulo 11 is 10
The multiplicative order of 8 modulo 11 is 10
The multiplicative order of 9 modulo 11 is 5
The multiplicative order of 10 modulo 11 is 2


Un théorème de Gauss stipule que, si $p$ est premier, alors il existe un entier $b$ dont l'ordre est précisément $p-1$ (le plus grand possible!). Un tel entier s'appelle une **racine primitive** modulo $p$. Par exemple, le calcul précédent a trouvé 12 racines primitives modulo $37$: 2, 5, 13, 15, 17, 18, 19, 20, 22, 24, 32, 35.
Pour illustrer ça (mod 37), consultez [cette affiche](https://fineartamerica.com/featured/epicycles-modulo-37-martin-weissman.html).

Pour tout ce qui suit, supposons que $p$ est un nombre premier. Non seulement des racines primitives *existent* mod $p$, mais elles sont assez courantes. En fait, le nombre de racines primitives mod $p$ est égal à $\phi(p-1)$, où $\phi$ indique l'[indicatrice d'Euler](https://fr.wikipedia.org/wiki/Indicatrice_d%27Euler). En moyenne, $\phi(n)$ correspond à environ $6 / \pi^2$ fois $n$ (pour les entiers positifs $n$). Bien que les nombres de la forme $p-1$ ne soient pas "moyens", on s’attend quand même à ce que $\phi(p-1)$ ne soit pas une très petite fraction de $p-1$.
Vous ne devriez pas avoir à chercher très loin si vous voulez *trouver* une racine primitive.

La partie la plus difficile, en pratique, consiste à déterminer si un nombre $b$ est ou non une racine primitive modulo $p$. Lorsque $p$ est très grand (comme des centaines ou des milliers de chiffres), $p-1$ est également très grand. Il n’est certainement pas pratique d’utiliser tous les puissances (de $1$ à  $p-1$) de $b$ pour déterminer si $b$ est une racine primitive!
La meilleure approche, parfois, consiste à utiliser le fait que l'ordre multiplicatif de $b$ doit être un **diviseur** de $p-1$. Si on peut trouver tous les diviseurs de $p-1$, il suffit de vérifier si $b^d \equiv 1$ mod $p$ pour chaque diviseur $d$. Cela rend le problème de déterminer si $b$ est une racine primitive à peu près aussi difficile que le problème de la factorisation de $p-1$. C'est un problème difficile, en général!

Mais pour l'application qui nous intéresse ici, nous voulons juste un grand nombre premier $p$ **et** une racine primitive mod $p$. Pour ce faire, le plus simple consiste à utiliser un nombre premier de **Sophie Germain** ; c'est un nombre premier $q$ tel que $2q + 1$ est également premier. Lorsque $q$ est un nombre premier de Sophie Germain, le nombre premier résultant $p = 2q + 1$ est appelé **nombre premier sûr**.

Observez que lorsque $p$ est un nombre premier sûr, la décomposition en facteurs premiers de de $p-1$ est
$p-1 = 2q$.

C'est tout. Ainsi, les ordres multiplicatifs possibles d'un élément $b$, mod $p$, sont les diviseurs de $2q$, qui sont $ 1, 2, q, 2q $
Pour vérifier si $b$ est une racine primitive, modulo un nombre premier sûr, $p = 2q + 1$, nous devons vérifier seulement trois choses: est-ce que $b \equiv 1$, ou $b^2 \equiv 1$, ou $b^q \equiv 1$, mod $p$? Si la réponse à ces trois questions est NON, alors $b$ est une racine primitive mod $p$.

In [3]:
def is_primroot_safe(b,p):
    '''
    Checks whether b is a primitive root modulo p,
    when p is a safe prime.  If p is not safe,
    the results will not be good!
    '''
    q = (p-1) // 2   # q is the Sophie Germain prime
    if b%p == 1:  # Is the multiplicative order 1?
        return False
    if (b*b)%p == 1:  # Is the multiplicative order 2?
        return False
    if pow(b,q,p) == 1:  # Is the multiplicative order q?
        return False
    return True  # If not, then b is a primitive root mod p.

In [8]:
q = 5
p = 2*q + 1
for b in range(2, p):
    print("{}    {}".format(b, is_primroot_safe(b, p)))

2    True
3    False
4    False
5    False
6    True
7    True
8    True
9    False
10    False


In [ ]:
b = 2

This would not be very useful if we couldn't find Sophie Germain primes.  Fortunately, they are not so rare.  The first few are 2, 3, 5, 11, 23, 29, 41, 53, 83, 89.  It is expected, but unproven that there are infinitely many Sophie Germain primes.  In practice, they occur fairly often.  If we consider numbers of magnitude $N$, about $1 / \log(N)$ of them are prime.  Among such primes, we expect about $1.3 / \log(N)$ to be Sophie Germain primes.  In this way, we can expect to stumble upon Sophie Germain primes if we search for a bit (and if $\log(N)^2$ is not too large).

The code below tests whether a number $p$ is a Sophie Germain prime.  We construct it by simply testing whether $p$ and $2p+1$ are both prime.  We use the Miller-Rabin test (the code from the previous Python notebook) in order to test whether each is prime.

In [ ]:
from random import randint # randint chooses random integers.

def Miller_Rabin(p, base):
    '''
    Tests whether p is prime, using the given base.
    The result False implies that p is definitely not prime.
    The result True implies that p **might** be prime.
    It is not a perfect test!
    '''
    result = 1
    exponent = p-1
    modulus = p
    bitstring = bin(exponent)[2:] # Chop off the '0b' part of the binary expansion of exponent
    for bit in bitstring: # Iterates through the "letters" of the string.  Here the letters are '0' or '1'.
        sq_result = result*result % modulus  # We need to compute this in any case.
        if sq_result == 1:
            if (result != 1) and (result != exponent):  # Note that exponent is congruent to -1, mod p.
                return False  # a ROO violation occurred, so p is not prime
        if bit == '0':
            result = sq_result 
        if bit == '1':
            result = (sq_result * base) % modulus
    if result != 1:
        return False  # a FLT violation occurred, so p is not prime.
    
    return True  # If we made it this far, no violation occurred and p might be prime.

def is_prime(p, witnesses=50):  # witnesses is a parameter with a default value.
    '''
    Tests whether a positive integer p is prime.
    For p < 2^64, the test is deterministic, using known good witnesses.
    Good witnesses come from a table at Wikipedia's article on the Miller-Rabin test,
    based on research by Pomerance, Selfridge and Wagstaff, Jaeschke, Jiang and Deng.
    For larger p, a number (by default, 50) of witnesses are chosen at random.
    '''
    if (p%2 == 0): # Might as well take care of even numbers at the outset!
        if p == 2:
            return True
        else:
            return False 
    
    if p > 2**64:  # We use the probabilistic test for large p.
        trial = 0
        while trial < witnesses:
            trial = trial + 1
            witness = randint(2,p-2) # A good range for possible witnesses
            if Miller_Rabin(p,witness) == False:
                return False
        return True
    
    else:  # We use a determinisic test for p <= 2**64.
        verdict = Miller_Rabin(p,2)
        if p < 2047:
            return verdict # The witness 2 suffices.
        verdict = verdict and Miller_Rabin(p,3)
        if p < 1373653:
            return verdict # The witnesses 2 and 3 suffice.
        verdict = verdict and Miller_Rabin(p,5)
        if p < 25326001:
            return verdict # The witnesses 2,3,5 suffice.
        verdict = verdict and Miller_Rabin(p,7)
        if p < 3215031751:
            return verdict # The witnesses 2,3,5,7 suffice.
        verdict = verdict and Miller_Rabin(p,11)
        if p < 2152302898747:
            return verdict # The witnesses 2,3,5,7,11 suffice.
        verdict = verdict and Miller_Rabin(p,13)
        if p < 3474749660383:
            return verdict # The witnesses 2,3,5,7,11,13 suffice.
        verdict = verdict and Miller_Rabin(p,17)
        if p < 341550071728321:
            return verdict # The witnesses 2,3,5,7,11,17 suffice.
        verdict = verdict and Miller_Rabin(p,19) and Miller_Rabin(p,23)
        if p < 3825123056546413051:
            return verdict # The witnesses 2,3,5,7,11,17,19,23 suffice.
        verdict = verdict and Miller_Rabin(p,29) and Miller_Rabin(p,31) and Miller_Rabin(p,37)
        return verdict # The witnesses 2,3,5,7,11,17,19,23,29,31,37 suffice for testing up to 2^64. 
    

In [ ]:
def is_SGprime(p):
    '''
    Tests whether p is a Sophie Germain prime
    '''
    if is_prime(p):  # A bit faster to check whether p is prime first.
        if is_prime(2*p + 1):  # and *then* check whether 2p+1 is prime.
            return True
    return False

Let's test this out by finding the Sophie Germain primes up to 100, and their associated safe primes.

In [ ]:
for j in range(1,100):
    if is_SGprime(j):
        print(j, 2*j+1)

Next, we find the first 100-digit Sophie Germain prime!  This might take a minute!

In [ ]:
test_number = 10**99 # Start looking at the first 100-digit number, which is 10^99.
while not is_SGprime(test_number):
    test_number = test_number + 1
print(test_number)

In the seconds or minutes your computer was running, it checked the primality of almost 90 thousand numbers, each with 100 digits.  Not bad!

### The Diffie-Hellman protocol

When we study protocols for secure communication, we must keep track of the communicating parties (often called Alice and Bob), and **who** has knowledge of **what** information.  We assume at all times that the "wire" between Alice and Bob is tapped -- anything they say to each other is actively monitored, and is therefore **public** knowledge.  We also assume that what happens on Alice's private computer is private to Alice, and what happens on Bob's private computer is private to Bob.  Of course, these last two assumptions are big assumptions -- they point towards the danger of computer viruses which infect computers and can violate such privacy!

The goal of the Diffie-Hellman protocol is -- at the end of the process -- for Alice and Bob to **share a secret** without ever having communicated the secret with each other.  The process involves a series of modular arithmetic calculations performed on each of Alice and Bob's computers.

The process begins when Alice or Bob creates and publicizes a **large prime number** `p` and a **primitive root** `g` modulo `p`.  It is best, for efficiency and security, to choose a **safe** prime `p`.  Alice and Bob can create their own safe prime, or choose one from a public list online, e.g., from the [RFC 3526 memo](https://tools.ietf.org/html/rfc3526).  Nowadays, it's common to take `p` with 2048 bits, i.e., a prime which is between $2^{2046}$ and $2^{2047}$ (a number with 617 decimal digits!).

For the purposes of this introduction, we use a smaller safe prime, with about 256 bits.  We use the `SystemRandom` functionality of the `random` package to create a good random prime.  It is not so much of an issue here, but in general one must be very careful in cryptography that one's "random" numbers are really "random"!  The `SystemRandom` function uses chaotic properties of your computer's innards in order to initialize a random number generator, and is considered cryptographically secure.

In [ ]:
from random import SystemRandom # Import the necessary package.

r = SystemRandom().getrandbits(256)
print("The random integer is {}".format(r))
print("with binary expansion {}".format(bin(r)))  #  r is an integer constructed from 256 random bits.
print("with bit-length {}.".format(len(bin(r)) - 2))  # In case you want to check.  Remember '0b' is at the beginning.

In [ ]:
def getrandSGprime(bitlength):
    '''
    Creates a random Sophie Germain prime p with about 
    bitlength bits.
    '''
    while True:
        p = SystemRandom().getrandbits(bitlength) # Choose a really random number.
        if is_SGprime(p):
            return p    

The function above searches and searches among random numbers until it finds a Sophie Germain prime.  The (possibly endless!) search is performed with a `while True:` loop that may look strange.  The idea is to stay in the loop until such a prime is found.  Then the `return p` command returns the found prime as output and halts the loop.  One must be careful with `while True` loops, since they are structured to run forever -- if there's not a loop-breaking command like `return` or `break` inside the loop, your computer will be spinning for a long time.

In [ ]:
q = getrandSGprime(256) # A random ~256 bit Sophie Germain prime
p = 2*q + 1 # And its associated safe prime

print("p is ",p)  # Just to see what we're working with.
print("q is ",q)

Next we find a primitive root, modulo the safe prime `p`.

In [ ]:
def findprimroot_safe(p):
    '''
    Finds a primitive root, 
    modulo a safe prime p.
    '''
    b = 2  # Start trying with 2.
    while True:  # We just keep on looking.
        if is_primroot_safe(b,p):
            return b
        b = b + 1 # Try the next base.  Shouldn't take too long to find one!

In [ ]:
g = findprimroot_safe(p)
print(g)

The pair of numbers $(g, p)$, the primitive root and the safe prime, chosen by either Alice or Bob, is now made public.  They can post their $g$ and $p$ on a public website or shout it in the streets.  It doesn't matter.  They are just tools for their secret-creation algorithm below.

### Alice and Bob's private secrets

Next, Alice and Bob invent **private** secret numbers $a$ and $b$.  They do not tell *anyone* these numbers.  Not each other.  Not their family.  Nobody.  They don't write them on a chalkboard, or leave them on a thumbdrive that they lose.  These are really secret.

But they don't use their phone numbers, or social security numbers.  It's best for Alice and Bob to use a secure random number generator on their separate private computers to create $a$ and $b$.  They are often 256 bit numbers in practice, so that's what we use below.

In [ ]:
a = SystemRandom().getrandbits(256)  # Alice's secret number
b = SystemRandom().getrandbits(256)  # Bob's secret number

print("Only Alice should know that a = {}".format(a))
print("Only Bob should know that b = {}".format(b))

print("But everyone can know p = {} and g = {}".format(p,g))

Now Alice and Bob use their secrets to generate new numbers.  Alice computes the number 
$$A = g^a \text{ mod } p,$$
and Bob computes the number
$$B = g^b \text{ mod } p.$$


In [ ]:
A = pow(g,a,p)  # This would be computed on Alice's computer.
B = pow(g,b,p)  # This would be computed on Bob's computer.

Now Alice and Bob do something that seems very strange at first.  Alice sends Bob her new number $A$ and Bob sends Alice his new number $B$.  Since they are far apart, and the channel is insecure, we can assume everyone in the world now knows $A$ and $B$.

In [ ]:
print("Everyone knows A = {} and B = {}.".format(A,B))

Now Alice, on her private computer, computes $B^a$ mod $p$.  She can do that because everyone knows $B$ and $p$, and she knows $a$ too.

Similarly, Bob, on his private computer, computes $A^b$ mod $p$.  He can do that because everyone knows $A$ and $p$, and he knows $b$ too.

Alice and Bob **do not** share the results of their computations!

In [ ]:
print(pow(B,a,p))  # This is what Alice computes.

In [ ]:
print(pow(A,b,p))  # This is what Bob computes.

Woah!  What happened?  In terms of exponents, it's elementary.  For
$$B^a = (g^{b})^a = g^{ba} = g^{ab} = (g^a)^b = A^b.$$
So these two computations yield the same result (mod $p$, the whole way through).

In the end, we find that Alice and Bob **share a secret**.  We call this secret number $S$.
$$S = B^a = A^b.$$

In [ ]:
S = pow(B,a,p)  # Or we could have used pow(A,b,p)
print(S)

This common secret $S$ can be used as a key for Alice and Bob to communicate hereafter.  For example, they might use $S$ (converted to a string, if needed) as the key for a Vigenère cipher, and chat with each other knowing that only they have the secret key to encrypt and decrypt their messages.

In [ ]:
#  We use the triple-quotes for a long string, that occupies multiple lines.
#  The backslash at the end of the line tells Python to ignore the newline character.
#  Imagine that Alice has a secret message she wants to send to Bob.  
#  She writes the plaintext on her computer. 

plaintext = '''Did you hear that the American Mathematical Society has an annual textbook sale? \
            It's 40 percent off for members and 25 percent off for everyone else.'''

In [ ]:
# Now Alice uses the secret S (as a string) to encrypt.  
ciphertext = Vigenere_cipher(plaintext, str(S))
print(ciphertext)
# Alice sends the following ciphertext to Bob, over an insecure channel.

In [ ]:
# When Bob receives the ciphertext, he decodes it with the secret S again.
print(Vigenere_decipher(ciphertext, str(S)))

To have confidence in this protocol, one needs to be convinced that their secret is truly secret!  The public has a lot of information:  they know 
1.  the prime $p$, 
2.  the primitive root $g$, 
3.  the number $A = g^a$ (mod $p$), and 
4.  the number $B = g^b$ (mod $p$).  

If the public could **figure out** either $a$ or $b$, they could figure out the secret (by raising $A^b$ or $B^a$ like Alice and Bob did).  

This is the essence of the **discrete logarithm problem**.  If we know the value of $g^a$ mod $p$, can we figure out the possible value(s) of $a$?  If this were ordinary arithmetic, we would say that $a = \log_g(A)$.  But this is modular arithmetic, and there's no easy way to figure out such logarithms.  The values of $g^a$ tend to bounce all over the place, mod $p$, especially since we chose $a$ to be pretty large (256 bits!).  

The security of the Diffie-Hellman protocol, i.e., the security of Alice and Bob's shared secret, depends on the *difficulty* of the discrete logarithm problem.  When $p$ is a large (e.g. 2048 bits) safe prime, and $a$ and $b$ are suitably large (roughly 256 bits), there seems to be no way to solve the discrete logarithm problem mod $p$ in any reasonable amount of time.  Someday we might have quantum computers to quickly solve discrete logarithm problems, and the Diffie-Hellman protocol will not be secure.  But for now, Alice and Bob's secret key seems safe.

### Exercises

1.  How many Sophie Germain primes are there between 1 and 1000000?  What proportion of primes in this range are Sophie Germain primes?

2.  [It is expected](https://en.wikipedia.org/wiki/Artin%27s_conjecture_on_primitive_roots) that there are infinitely primes $p$ such that 2 is a primitive root mod $p$.  Study the density of such primes.  For example, among the primes up to 1000, how often is $2$ a primitive root?  Does this density seem to change?  

3.  Adapt Diffie-Hellman to work with a group of *three* parties who wish to share a common secret.  Hint:  the common secret will have the form $g^{abc}$, and other exponents like $g^a$, $g^b$, $g^c$, $g^{ab}$, $g^{bc}$, $g^{ac}$ will be public information.

4.  Sadly, Alice and Bob have agreed to use the primitive root $g = 3$ and the prime $p = 65537$.  Listening into their conversation, you intercept the following: $A = 40360$ and $B = 21002$ and the ciphertext is `$;6HWD;P5LVJ99W+EH9JVx=I:V7ESpGC^`.  If you know that they use a protocol with a Vigenère cipher, with key equal the string associated to their secret $S$, what is the plaintext message?  Hint:  you should be able to solve the discrete logarithm problem with a brute force attack.

